In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
HRD_file = pd.read_excel("../HistoFL-master/DATA_ROOT_DIR/HRD_Score.xlsx")

In [3]:
HRD_file_BRCA = HRD_file[HRD_file['cancer type']=='BRCA']
label = ['patient ID', 'HRD_Status']
BRCA_data=pd.DataFrame()
for i in label:
    BRCA_data[str(i)] = HRD_file_BRCA [i] 
print(BRCA_data)

           patient ID  HRD_Status
476   TCGA-3C-AAAU-01         0.0
477   TCGA-3C-AALI-01         0.0
478   TCGA-3C-AALJ-01         0.0
479   TCGA-3C-AALK-01         0.0
480   TCGA-4H-AAAK-01         0.0
...               ...         ...
1452  TCGA-WT-AB44-01         0.0
1453  TCGA-XX-A899-01         0.0
1454  TCGA-XX-A89A-01         0.0
1455  TCGA-Z7-A8R5-01         0.0
1456  TCGA-Z7-A8R6-01         0.0

[981 rows x 2 columns]


In [7]:
# 单站点数据生成
h5_lists = os.listdir("../HistoFL-master/DATA_ROOT_DIR/classification_features_dir/h5_files")
slide_lists = []

slide_data = pd.DataFrame()
for file in h5_lists:
    basename, _ = os.path.splitext(file)
    h5_patient = basename[len('slide_'):len('slide_') + 15]
    if h5_patient in BRCA_data['patient ID'].values:
        BRCA_index = BRCA_data[BRCA_data['patient ID']==h5_patient].index[0]
        new_row = pd.DataFrame([{
            "case_id": h5_patient, 
            "slide_id": basename, 
            "censorship": "class_" + str(int(BRCA_data.at[BRCA_index, 'HRD_Status'])), 
            "institute": "site_0"
        }])
        slide_data = pd.concat([slide_data, new_row], ignore_index=True)
print(slide_data.shape)
print(slide_data)


(1012, 4)
              case_id                                           slide_id  \
0     TCGA-EW-A1J6-01  slide_TCGA-EW-A1J6-01Z-00-DX1.F668E978-13B6-42...   
1     TCGA-A2-A0T3-01  slide_TCGA-A2-A0T3-01Z-00-DX1.5E96BC87-F4FB-4A...   
2     TCGA-C8-A27A-01  slide_TCGA-C8-A27A-01Z-00-DX1.0E26C46D-CD65-40...   
3     TCGA-AR-A0U4-01  slide_TCGA-AR-A0U4-01Z-00-DX1.DE722DC5-859D-48...   
4     TCGA-BH-A1EW-01  slide_TCGA-BH-A1EW-01Z-00-DX1.A6A5F9C5-FB97-4D...   
...               ...                                                ...   
1007  TCGA-E2-A574-01  slide_TCGA-E2-A574-01Z-00-DX1.60341091-B118-4F...   
1008  TCGA-D8-A1Y3-01  slide_TCGA-D8-A1Y3-01Z-00-DX2.B9BD14AF-52A6-42...   
1009  TCGA-AN-A0AS-01  slide_TCGA-AN-A0AS-01Z-00-DX1.51E551E9-E5F4-4C...   
1010  TCGA-BH-A0EI-01  slide_TCGA-BH-A0EI-01Z-00-DX1.929E126A-C9F8-42...   
1011  TCGA-BH-A18G-01  slide_TCGA-BH-A18G-01Z-00-DX1.DB2B5819-CE83-4E...   

     censorship institute  
0       class_0    site_0  
1       class_0    si

In [42]:
# 多站点数据生成
site_nums = 2
h5_lists = os.listdir("../HistoFL-master/DATA_ROOT_DIR/classification_features_dir/h5_files")
slide_lists = []
slide_data = pd.DataFrame()
slide_data_init = pd.DataFrame()
for file in h5_lists:
    basename, _ = os.path.splitext(file)
    h5_patient = basename[len('slide_'):len('slide_') + 15]
    if h5_patient in BRCA_data['patient ID'].values:
        BRCA_index = BRCA_data[BRCA_data['patient ID']==h5_patient].index[0]
        new_row = pd.DataFrame([{
            "case_id": h5_patient, 
            "slide_id": basename, 
            "censorship": "class_" + str(int(BRCA_data.at[BRCA_index, 'HRD_Status'])), 
            "institute": ""
        }])
        slide_data_init = pd.concat([slide_data_init, new_row], ignore_index=True)
# 将slide_data按类别分组
grouped = slide_data_init.groupby('censorship')
# 初始化一个空的列表来存储每个站点的数据
site_dataframes = [[] for _ in range(site_nums)]
# 按类别将数据分配到各个站点
for _, group in grouped:
    # 将每个类别的数据按站点数量分块
    chunks = np.array_split(group, site_nums)
    for idx, chunk in enumerate(chunks):
        site_dataframes[idx].append(chunk)
        chunk['institute'] = f"site_{idx}"
        slide_data = pd.concat([slide_data, chunk], ignore_index=True)
# print(slide_data)
# 输出当前site_data中site0的个数以及site1的个数
site0_count = len(slide_data[slide_data['institute'] == 'site_0'])
site1_count = slide_data[slide_data['institute'] == 'site_1'].shape[0]

# 输出site0中class_0的个数
site0_class0_count = slide_data[(slide_data['institute'] == 'site_0') & (slide_data['censorship'] == 'class_0')].shape[0]
site0_class1_count = slide_data[(slide_data['institute'] == 'site_0') & (slide_data['censorship'] == 'class_1')].shape[0]
# 输出site1中class_0的个数
site1_class0_count = slide_data[(slide_data['institute'] == 'site_1') & (slide_data['censorship'] == 'class_0')].shape[0]
site1_class1_count = slide_data[(slide_data['institute'] == 'site_1') & (slide_data['censorship'] == 'class_1')].shape[0]
print(f"site_0 count: {site0_count}")
print(f"site_1 count: {site1_count}")
print(f"site_0 class_0 count: {site0_class0_count}")
print(f"site_0 class_1 count: {site0_class1_count}")
print(f"site_1 class_0 count: {site1_class0_count}")
print(f"site_1 class_1 count: {site1_class1_count}")
# slide_data.to_csv("tmp.csv", index=False)
# slide_data.to_csv("./dataset_csv/BRCA_fl_data.csv", index=False)

site_0 count: 507
site_1 count: 505
site_0 class_0 count: 413
site_0 class_1 count: 94
site_1 class_0 count: 412
site_1 class_1 count: 93


/home/lzhao/anaconda/envs/clam_latest/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [47]:
# 划分数据集
from sklearn.model_selection import train_test_split
def split_data_by_site(slide_data, site_nums):
    res_df = pd.DataFrame()
    for site_id in range(site_nums):
        df_cur = slide_data[slide_data['institute'] == f"site_{site_id}"]['slide_id']
        df_cur = df_cur.reset_index(drop=True)
        train, temp = train_test_split(df_cur, test_size=0.2, random_state=42)
        train= train.reset_index(drop=True)
        # Split temp into validation and test
        valid, test = train_test_split(temp, test_size=0.5, random_state=42)
        valid= valid.reset_index(drop=True)
        test= test.reset_index(drop=True)

        tmp ={"train":train, "val":valid, "test" : test}
        df_col = pd.DataFrame(tmp)
        print(df_col.shape)
        # print(df_col)
        # df_col.to_csv("00.csv")
        res_df = pd.concat([res_df, df_col], ignore_index=True) 
        res_df.to_csv(f"tmp.csv", index=False)
    return res_df

# site_nums = len(slide_data['institute'].dropna().unique())
# print(site_nums)
# split_data_by_site(slide_data, site_nums)

input_path = "./dataset_csv/"
file = "BRCA_fl_data.csv"
site_nums = len(pd.read_csv(input_path + file)['institute'].dropna().unique())
print("site_nums:", site_nums)
# file = "BRCA_nofl_data.csv"
split_train_test_valid(input_path, file, site_nums)

site_nums: 2
(507,)
(405, 3)
(505,)
(404, 3)


,train,val,test
0,slide_TCGA-A7-A0CJ-01Z-00-DX1.E26F2F62-D688-43...,slide_TCGA-EW-A1J6-01Z-00-DX1.F668E978-13B6-42...,slide_TCGA-A8-A06T-01Z-00-DX1.BA8B8FC3-6169-48...
1,slide_TCGA-B6-A0WT-01Z-00-DX1.7ED21DA2-A3D8-4D...,slide_TCGA-A8-A0A4-01Z-00-DX1.06C38F98-F34B-49...,slide_TCGA-EW-A2FS-01Z-00-DX1.A01C9183-2AC8-45...
2,slide_TCGA-AR-A254-01Z-00-DX1.EA5CD008-1106-41...,slide_TCGA-AN-A0AM-01Z-00-DX1.169CE39A-DD54-46...,slide_TCGA-BH-A0BD-01Z-00-DX1.CD4A6FC2-BA8C-4E...
3,slide_TCGA-LL-A9Q3-01Z-00-DX1.94FA76F5-008A-40...,slide_TCGA-EW-A2FW-01Z-00-DX1.4F948681-81F3-46...,slide_TCGA-BH-A0HI-01Z-00-DX1.89DB9F79-0DCC-4C...
4,slide_TCGA-C8-A274-01Z-00-DX1.B667DAB1-C944-42...,slide_TCGA-D8-A13Y-01Z-00-DX1.02321E77-A11E-41...,slide_TCGA-AO-A1KO-01Z-00-DX1.EEB5E0A0-92B2-42...
...,...,...,...
804,slide_TCGA-D8-A1JD-01Z-00-DX2.C6A7667E-25C9-48...,NaN,NaN
805,slide_TCGA-BH-A18H-01Z-00-DX1.4EC9108F-04C2-4B...,NaN,NaN
806,slide_TCGA-GM-A2DA-01Z-00-DX1.6E409F88-F654-48...,NaN,NaN
807,slide_TCGA-D8-A1XW-01Z-00-DX2.9849E503-BE3E-41...,NaN,NaN


In [53]:
input_path = "./dataset_csv/"
# file = "classification_hrd_dataset_fl.csv"
# file = "classification_hrd_dataset.csv"
# file = "BRCA_fl_data.csv"
file = "BRCA_nofl_data.csv"
k = 5
site_nums = len(pd.read_csv(input_path + file)['institute'].dropna().unique())
print("site_nums:", site_nums)
for kindex in range(k):
    res = split_train_test_valid(input_path, file, site_nums)
    print(f"train count: {res['train'].dropna().shape[0]}")
    print(f"val count: {res['val'].dropna().shape[0]}")
    print(f"test count: {res['test'].dropna().shape[0]}")
    # if not os.path.exists("./tmp/nofl_classification"):
    #     os.makedirs("./tmp/nofl_classification")
    # res.to_csv("./tmp/nofl_classification/splits_{}.csv".format(kindex))
    if not os.path.exists("./splits/fl_classification"):
        os.makedirs("./splits/fl_classification")
    res.to_csv("./splits/fl_classification/splits_{}.csv".format(kindex))

site_nums: 1
(1012,)
(809, 3)
train count: 809
val count: 101
test count: 102
(1012,)
(809, 3)
train count: 809
val count: 101
test count: 102
(1012,)
(809, 3)
train count: 809
val count: 101
test count: 102
(1012,)
(809, 3)
train count: 809
val count: 101
test count: 102
(1012,)
(809, 3)
train count: 809
val count: 101
test count: 102


In [31]:
# ----------删-----------------
input_path = "./dataset_csv/"
# file = "classification_hrd_dataset_fl.csv"
# file = "classification_hrd_dataset.csv"
# file = "BRCA_fl_data.csv"
file = "BRCA_nofl_data.csv"
k = 5
site_nums = len(pd.read_csv(input_path + file)['institute'].dropna().unique())
print("site_nums:", site_nums)
for kindex in range(k):
    res = split_train_test_valid(input_path, file, site_nums)
    for index, row in res.iterrows():
        site_name = row['site']
        site_data = row['data']
        # site_data.to_csv("00.csv")
        # print(site_data)
        # if not os.path.exists("./splits/fl_classification".format(site_name)):
        #     os.makedirs("./splits/fl_classification/site_{}".format(site_name))
        # -------------correct-----------------
        # if not os.path.exists("./splits/nofl_classification".format(site_name)):
        #     os.makedir("./splits/nofl_classification".format(site_name))
        print(kindex, "次划分:", site_name, ":train", len(site_data["train"].dropna()), ",val", len(site_data["val"].dropna()), ",test", len(site_data["test"].dropna()))
        # site_data.to_csv("./splits/nofl_classification/splits_{}.csv".format(kindex))

site_nums: 1
(1012,)
(809, 3)


KeyError: 'site'